# Preprocess Dataset/Generate Dataset

In [25]:
import os
import torch
import pathlib
import shutil
import numpy as np
import PIL
from fastai.vision import *
from torchvision import transforms

In [53]:
DIV2K_path = pathlib.Path('.').parent.absolute()
train_HR = DIV2K_path/'DIV2K_train_HR'
valid_HR = DIV2K_path/'DIV2K_valid_HR'
assert DIV2K_path.exists() and train_HR.exists() and valid_HR.exists()

In [54]:
high_res = 600
low_res_factor = 4
low_res = high_res // low_res_factor
output_path = DIV2K_path/'custom'/f'{low_res}x4'
print(f'output path: {output_path}')

output path: /home/hacker/Documents/Super-Resolution/datasets/DIV2K/custom/150x4


In [55]:
# clean and create output_path
if output_path.exists():
    shutil.rmtree(output_path)
output_path.mkdir(parents=True, exist_ok=True)

In [56]:
# create subfolders
hr_train = output_path/'hr_train'
hr_valid = output_path/'hr_valid'
lr_train = output_path/'lr_train'
lr_valid = output_path/'lr_valid'
for dir_path in [hr_train, hr_valid, lr_train, lr_valid]:
    dir_path.mkdir(parents=True, exist_ok=True)

## Get Image Info

In [9]:
hr_valid_image_list = ImageList.from_folder(valid_HR)
hr_train_image_list = ImageList.from_folder(train_HR)

In [10]:
hr_train_image_name_list = [img_path.relative_to(train_HR) for img_path in hr_train_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in hr_train_image_list.items]
print(f"min dimension of all training images={torch.min(torch.tensor(shapes))}")
hr_valid_image_name_list = [img_path.relative_to(valid_HR) for img_path in hr_valid_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in hr_valid_image_list.items]
print(f"min dimension of all validation images={torch.min(torch.tensor(shapes))}")

min dimension of all training images=648
min dimension of all validation images=816


In [7]:
def crop_HR_image_and_save(src_path, dest_path, image_name_list):
    for image_name in image_name_list:
        src_image_path = src_path/image_name
        target_image_path = dest_path/image_name
        transformed_img = HR_crop_transforms(PIL.Image.open(src_image_path))
        transformed_img.save(target_image_path)

## Image Agumentation

In [106]:
def get_proper_size_center_crop(image, size: int):
    return transforms.Compose([
        transforms.CenterCrop(min(image.size)),
        transforms.Resize(size, interpolation=PIL.Image.BICUBIC)
    ])(image)

In [ ]:
# transform and save images
def test(imagename):
    print(imagename)
parallel(test, valid_image_name_list)
# parallel(ImageAugmentor(train_HR, train_HR_crop, high_res), train_image_name_list)

In [123]:
class ImageAugmentor(object):
    def __init__(self, src_path, dest_dir, size: int, num_extra:int=2):
        assert num_extra <= 5
        self.num_extra = num_extra
        self.src_path = src_path
        self.dest_dir = dest_dir
        self.size = size
        self.image_name = os.path.basename(src_path)
        self.filename_no_ext, self.ext = os.path.splitext(self.image_name)
        # get base image
        img = PIL.Image.open(src_path)
        self.base_image = get_proper_size_center_crop(img, size)
        self.generated_images = [self.base_image]
        self.generated_image_names = [self.image_name]


    def generate(self):
        random_choices = np.random.choice(a=[1, 2, 3, 4, 5], replace=False, size=self.num_extra)
        print(random_choices)
        for choice in random_choices:
            if choice == 1:
                self.generated_images.append(transforms.functional.hflip(self.base_image))                
                self.generated_image_names.append(self.filename_no_ext + '-hf' + self.ext)
            elif choice == 2:
                self.generated_images.append(transforms.functional.vflip(self.base_image))
                self.generated_image_names.append(self.filename_no_ext + '-vf' + self.ext)
            elif choice == 3:
                self.generated_images.append(transforms.functional.rotate(self.base_image, 90))
                self.generated_image_names.append(self.filename_no_ext + '-r90' + self.ext)
            elif choice == 4:
                self.generated_images.append(transforms.functional.rotate(self.base_image, 180))
                self.generated_image_names.append(self.filename_no_ext + '-r180' + self.ext)
            elif choice == 5:
                self.generated_images.append(transforms.functional.rotate(self.base_image, 270))
                self.generated_image_names.append(self.filename_no_ext + '-r270' + self.ext)

    def save(self):
        for i in range(len(self.generated_image_names)):
            output_path = self.dest_dir/self.generated_image_names[i]
            self.generated_images[i].save(output_path)



In [ ]:
class ImageAugmentorWrapper(object):
    def __init__(self, src_path, dest_path, size, num_extra:int=2):
        self.src_path = src_path
        self.dest_path = dest_path
        self.size = size
        self.num_extra = num_extra

    def __call__(self, image_name, i):
        src_image_path = self.src_path/image_name
        target_image_path = self.dest_path/image_name
        IA = ImageAugmentor(src_path, dest_dir, size: int, num_extra=self.num_extra)
        transformed_img.save(target_image_path)

In [17]:
img = PIL.Image.open('/home/hacker/Documents/Super-Resolution/datasets/DIV2K/DIV2K_train_HR/0003.png')

In [124]:
IA = ImageAugmentor('/home/hacker/Documents/Super-Resolution/datasets/DIV2K/DIV2K_train_HR/0003.png', pathlib.Path('/home/hacker/Documents/Super-Resolution/datasets/DIV2K/custom/150x4/hr_train'), 600)
IA.generate()
IA.save()

[3 1]


In [8]:
class ImageAug:
    def __init__(self, image, size, five_crop=True, n_rand_crop=1, flip=True, rotate=True, angles=[90, 180, 270], hflip=True, vflip=True):
        self.size =size
        self.angles = angles
        self.original_image = image
        self.cropped_images = []
        self.transformed_images = []
        self.five_crop = five_crop
        self.n_rand_crop = n_rand_crop
        self.hflip = hflip
        self.vflip = vflip

    def crop(self):
        # five crop
        if self.five_crop:
            self.cropped_images.extend(transforms.Compose([
                transforms.FiveCrop(self.size)
            ])(self.original_image))
        # random crop
        for _ in range(n_rand_crop):
            self.cropped_images.append(transforms.Compose([
                transforms.RandomCrop(min(self.original_image.size)),
                transforms.RandomCrop(self.size, pad_if_needed=True, padding_mode='reflect'),
            ])(self.original_image))

        
    def transform(self):
        # transform based on existing
        for image in self.cropped_images:
            # rotate
            for angle in self.angles:
                self.transformed_images.append(transforms.functional.rotate(image, angle))
            # flip
            if self.vflip:
                self.transformed_images.append(transforms.functional.vflip(image))
            if self.hflip:
                self.transformed_images.append(transforms.functional.hflip(image))


In [24]:
class HR_Cropper(object):
    def __init__(self, src_path, dest_path, high_res):
        self.src_path = src_path
        self.dest_path = dest_path
        self.high_res = high_res

    def __call__(self, image_name, i):
        src_image_path = self.src_path/image_name
        target_image_path = self.dest_path/image_name
        src_img = PIL.Image.open(src_image_path)
        transformed_img = transforms.Compose([
            transforms.RandomCrop(min(src_img.size)),
            transforms.RandomCrop(self.high_res, pad_if_needed=True, padding_mode='reflect'),
        ])(src_img)
        transformed_img.save(target_image_path)

In [25]:
# crop and save images
parallel(HR_Cropper(valid_HR, valid_HR_crop, high_res), valid_image_name_list)
parallel(HR_Cropper(train_HR, train_HR_crop, high_res), train_image_name_list)

In [26]:
# Cropped Train Images
train_HR_crop_image_list = ImageList.from_folder(train_HR_crop)
train_HR_crop_image_name_list = [img_path.relative_to(train_HR_crop) for img_path in train_HR_crop_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in train_HR_crop_image_list.items]
print(f"min dimension of all images={torch.min(torch.tensor(shapes))}")

min dimension of all images=600


In [27]:
# Cropped Train Images
valid_HR_crop_image_list = ImageList.from_folder(valid_HR_crop)
valid_HR_crop_image_name_list = [img_path.relative_to(valid_HR_crop) for img_path in valid_HR_crop_image_list.items]
shapes = [PIL.Image.open(img_path).size for img_path in valid_HR_crop_image_list.items]
print(f"min dimension of all images={torch.min(torch.tensor(shapes))}")

min dimension of all images=600


## Generate Low Resolution Data

In [28]:
low_res_size = high_res // low_res_factor
print(f"low_res_size={low_res_size}")

low_res_size=150


In [29]:
# create destination directory
train_LR = DIV2K_path/'custom'/f'DIV2K_train_LR_{high_res}_{low_res_size}'
train_LR.mkdir(parents=True, exist_ok=True)
valid_LR = DIV2K_path/'custom'/f'DIV2K_valid_LR_{high_res}_{low_res_size}'
valid_LR.mkdir(parents=True, exist_ok=True)

In [30]:
class Downscaler(object):
    def __init__(self, src_path, dest_path, low_res_size):
        self.src_path = src_path
        self.dest_path = dest_path
        self.low_res_size = low_res_size

    def __call__(self, image_name, i):
        src_image_path = self.src_path/image_name
        target_image_path = self.dest_path/image_name
        src_img = PIL.Image.open(src_image_path)
        downscaled_img = src_img.resize((self.low_res_size, self.low_res_size), resample=PIL.Image.BILINEAR).convert('RGB')
        downscaled_img.save(target_image_path)

In [31]:
# transform and save images
parallel(Downscaler(valid_HR_crop, valid_LR, low_res_size), valid_HR_crop_image_name_list)
parallel(Downscaler(train_HR_crop, train_LR, low_res_size), train_HR_crop_image_name_list)